In [ ]:
import os
import mediapipe as mp
import cv2
import numpy as np1


folder_path = r"/home/rambo/Desktop/test/yawn1"  # Thay đổi đường dẫn đến thư mục của bạn
output_folder = r"/home/rambo/Desktop/test/yawn_crop"


mp_eye_landmarks = mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1)
Mouth =[2,206,212,211,171,400,431,432,426]
eye = [6,119,143,156,52,55,168,295,276,340,357]
eye_mouth = [2,206,212,211,171,400,431,432,426,6,119,143,156,52,55,168,295,276,340,357]
# Lặp qua tất cả các tệp trong thư mục
for filename in os.listdir(folder_path):
    # Kiểm tra xem tệp có phải là tệp .jpg không
    if filename.endswith(".jpg"):
        img_path = os.path.join(folder_path, filename)
        image = cv2.imread(img_path)
    # Chuyển đổi ảnh sang không gian màu RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Nhận diện landmarks của khuôn mặt trên ảnh
        results = mp_eye_landmarks.process(image_rgb)

        # Kiểm tra xem có landmarks nào được nhận diện không
        if results.multi_face_landmarks:
            # Lấy danh sách các landmarks của khuôn mặt đầu tiên trong ảnh
            face_landmarks = results.multi_face_landmarks[0]

            # Tạo danh sách tọa độ của các điểm LEFT_EYE
            eye_mouth_lanmarks = []
            eye_lanmarks = []
            min_x1 = 1000
            min_y1 = 1000
            max_x1 = 0
            max_y1 = 0
            for index in eye_mouth:
                landmark = face_landmarks.landmark[index]
                landmark_x = int(landmark.x * image.shape[1])
                landmark_y = int(landmark.y * image.shape[0])
                min_x1 = min(landmark_x,min_x1)
                min_y1 = min(landmark_y,min_y1)
                max_x1 = max(landmark_x,max_x1)
                max_y1 = max(landmark_y,max_y1)

                eye_mouth_lanmarks.append((landmark_x, landmark_y))
            # print(eye_mouth_lanmarks[0][0])
            # Tạo mask để làm nổi bật vùng khuôn mặt
            mask = np.zeros(image.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [np.array(eye_mouth_lanmarks[:9])], (255, 255, 255))
            cv2.fillPoly(mask, [np.array(eye_mouth_lanmarks[9:21])], (255, 255, 255))

            # Áp dụng mask để chỉ hiển thị vùng khuôn mặt
            result = cv2.bitwise_and(image, image, mask=mask)
            result = cv2.resize(result[int(min_y1):int(max_y1), int(min_x1):int(max_x1)],(224,224))
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path,result)